In [53]:
import warnings
warnings.filterwarnings("ignore")

In [54]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
import time
import pyautogui

In [68]:
# Move to mouse 

pyautogui.moveTo(1500,400,duration=1)

pyautogui.FAILSAFE=False

driver_path = "/usr/local/bin/chromedriver"

main_url = "https://www.hepsiburada.com/"

# Create an empty DataFrame for user reviews and products

reviews_df = pd.DataFrame(columns=['user reviews', 'rating', 'product url'])

products_df = pd.DataFrame(columns=['product url', 'product name', 'img url','brand name','category name'])

service = ChromeService(executable_path=driver_path)

driver = webdriver.Chrome(service=service)

driver.get(main_url)

for j in range(0,18,2):

    # Get the category name

    action = ActionChains(driver)

    navbar_element = driver.find_element(By.CLASS_NAME,'sf-MenuItems-WulWXvlfIAwNiOUGY7FP') 

    driver.execute_script("arguments[0].scrollIntoView();", navbar_element)

    action.move_to_element(navbar_element).perform()

    time.sleep(4)
    
    category_tag = driver.find_elements(By.CLASS_NAME,'sf-ChildMenuItems-a4G0z3YJJWkQs7qKKuuj')[j]

    category_name = category_tag.text

    category_tag.click()

    time.sleep(5)

    # Sort the products from those with the most comments to fewest 

    order_button = driver.find_elements(By.CSS_SELECTOR,"div[data-test-id='dropdown-container']")[1]

    driver.execute_script("arguments[0].scrollIntoView();", order_button)

    order_button.click()

    time.sleep(5)

    sort_by_reviews = driver.find_elements(By.CLASS_NAME,'horizontalSortingBar-PkoDOH7UsCwBrQaQx9bn')

    sort_by_reviews[3].click()

    driver.refresh()

    # Find <li> tags containing the list of products

    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CLASS_NAME,"productListContent-zAP0Y5msy8OHn5z7T_K_"))
    )

    li_tags = driver.find_elements(By.CLASS_NAME,"productListContent-zAP0Y5msy8OHn5z7T_K_")

    # Loop through each product

    for li in li_tags:
        product_url = None
        product_name = None
        img_url = None
        user_review = None
        rating = None
        brand_name = None
        
        # Get the product URL
        product_url = li.find_element(By.TAG_NAME,"a").get_attribute("href")

        # Open a new window, Switch to the new window
        driver.execute_script("window.open('');")

        driver.switch_to.window(driver.window_handles[-1]) 

        driver.get(product_url)

        driver.set_window_size(1200, 900)

        # Get the product name
        product_name = driver.find_element(By.ID,"product-name").text

        # Get the brand name 
        brand_name = driver.find_element(By.XPATH,"/html/body/div[2]/main/div[3]/section[1]/div[3]/div/div[4]/div[1]/span/a").text

        # Get the product image URL
        img_url = driver.find_element(By.CSS_SELECTOR,"img[class='product-image']").get_attribute("src")

        # Scroll down the page, Wait for user reviews to load
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

        driver.find_element(By.XPATH,'//*[@id="productReviewsTab"]').click()

        # Process each user review
        try:
            all_reviews_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//*[@id='hermes-voltran-comments']/div[2]/div[1]/a"))).click()

            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

            try:
                last_reviews = driver.find_elements(By.XPATH,'//*[@id="hermes-voltran-comments"]/div[6]/div[3]/div[2]/div/ul/li')[-1].text
            except IndexError:
                last_reviews = 1

            driver.set_window_size(400, 826)

            for i in range(int(last_reviews)):

                try:

                    all_reviews_div = WebDriverWait(driver, 5).until(
                        EC.presence_of_all_elements_located((By.CLASS_NAME,"hermes-ReviewCard-module-BJtQZy5Ub3goN_D0yNOP")))

                    WebDriverWait(driver, 5).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[class='star']")))

                    WebDriverWait(driver, 5).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span[itemprop='description']")))

                except:

                    pass

                for review in all_reviews_div:

                    try:
                        rating = len(review.find_elements(By.CSS_SELECTOR, "div[class='star']"))
                    except:
                        rating = None

                    try:
                        user_review = review.find_element(By.CSS_SELECTOR, "span[itemprop='description']").text

                    except:
                        user_review = None

                    reviews_df.loc[len(reviews_df)] = [user_review,rating,product_url]

                next_page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
                    (By.XPATH,'//*[@id="hermes-voltran-comments"]/div[6]/div[3]/div[2]/div/div/div/div[3]')))
                
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                next_page_button.click()
                
            driver.close()

            # Switch back to the main window
            driver.switch_to.window(driver.window_handles[0])

            # Add products to the DataFrame
            products_df.loc[len(products_df)]=[product_url,product_name,img_url,brand_name,category_name]

        except TimeoutException:

            driver.close()
            
            # Switch back to the main window
            driver.switch_to.window(driver.window_handles[0])
            continue

    driver.set_window_size(1200, 900)

driver.quit()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=121.0.6167.184)
Stacktrace:
0   chromedriver                        0x000000010096a7dc chromedriver + 4040668
1   chromedriver                        0x00000001009629e0 chromedriver + 4008416
2   chromedriver                        0x00000001005d5870 chromedriver + 284784
3   chromedriver                        0x00000001005be0d8 chromedriver + 188632
4   chromedriver                        0x00000001005be004 chromedriver + 188420
5   chromedriver                        0x0000000100652778 chromedriver + 796536
6   chromedriver                        0x000000010060d74c chromedriver + 513868
7   chromedriver                        0x000000010060e044 chromedriver + 516164
8   chromedriver                        0x000000010092fa04 chromedriver + 3799556
9   chromedriver                        0x0000000100933ee4 chrome

In [64]:
products_df

,product url,product name,img url,brand name,category name
0,https://www.hepsiburada.com/gakademi-goruntulu...,GAkademi Görüntülü Online Eğitim Platformu,https://productimages.hepsiburada.net/s/43/550...,Görüntülü Akademi,Bilgisayar/Tablet
1,https://www.hepsiburada.com/xiaomi-mi-wifi-pro...,Xiaomi Mi Wifi Pro Sinyal Yakınlaştırıcı - Güç...,https://productimages.hepsiburada.net/s/36/550...,Xiaomi,Bilgisayar/Tablet
2,https://www.hepsiburada.com/hp-650-siyah-murek...,HP 650 Siyah Mürekkep Kartuşu (CZ101AE),https://productimages.hepsiburada.net/s/181/55...,HP,Yazıcılar & Projeksiyon
3,https://www.hepsiburada.com/hp-304-siyah-murek...,HP 304 Siyah Mürekkep Kartuşu N9K06AE,https://productimages.hepsiburada.net/s/6/550/...,HP,Yazıcılar & Projeksiyon
4,https://www.hepsiburada.com/apple-20-w-usb-c-g...,Apple 20 W USB-C Güç Adaptörü - MHJE3TU/A (App...,https://productimages.hepsiburada.net/s/48/550...,Apple,Telefon & Telefon Aksesuarları
5,https://www.hepsiburada.com/iphone-11-128-gb-p...,iPhone 11 128 GB,https://productimages.hepsiburada.net/s/49/550...,Apple,Telefon & Telefon Aksesuarları
6,https://www.hepsiburada.com/xiaomi-mi-box-s-4k...,Xiaomi Mi Box S 4K Android TV Box,https://productimages.hepsiburada.net/s/22/550...,Xiaomi,"TV, Görüntü & Ses Sistemleri"
7,https://www.hepsiburada.com/arbalest-deep-clea...,"Arbalest Deep Cleaning Ocak, Fırın & Yüzey Tem...",https://productimages.hepsiburada.net/s/96/550...,Arbalest,Beyaz Eşya
8,https://www.hepsiburada.com/kumtel-antrasit-gr...,Kumtel Antrasit ( Gri ) Cam Digital / Dokunmat...,https://productimages.hepsiburada.net/s/122/55...,Kumtel,Beyaz Eşya
9,https://www.hepsiburada.com/kumtel-lx-6331-2in...,Kumtel LX-6331 (2ın1) 2000W - Sıcak & Soğuk Ha...,https://productimages.hepsiburada.net/s/49/550...,Kumtel,Klima ve Isıtıcılar


In [65]:
reviews_df

,user reviews,rating,product url
0,Ürünün 180 günlük sınırlı bir süre ile gelmesi...,3,https://www.hepsiburada.com/gakademi-goruntulu...
1,Pek bi işe yaradığını düşünmüyorum keşke bunun...,5,https://www.hepsiburada.com/gakademi-goruntulu...
2,"Güzel bir hediye ancak, keşke kargolamak yerin...",5,https://www.hepsiburada.com/gakademi-goruntulu...
3,"Hepsiburada ürünümü göndermeyip, ayrıca müşter...",5,https://www.hepsiburada.com/gakademi-goruntulu...
4,Amerikan English Kursu 8 kuru bu fiyata veriyo...,5,https://www.hepsiburada.com/gakademi-goruntulu...
...,...,...,...
175,İndirimli aldığım için çok memnunum ses biraz ...,4,https://www.hepsiburada.com/sup-tasinabilir-40...
176,"Çok eğlenceli bir konsol.Içinde mario,tetris,a...",4,https://www.hepsiburada.com/sup-tasinabilir-40...
177,"Sorunsuz geldi teşekkürler, çok güzel bir ciha...",5,https://www.hepsiburada.com/sup-tasinabilir-40...
178,Ürün iyi ama aşırı bi beklenti olmasın ekran r...,4,https://www.hepsiburada.com/sup-tasinabilir-40...


In [66]:
len(reviews_df["user reviews"].unique())

169

In [67]:
len(reviews_df[reviews_df["user reviews"].isna()])

1

In [61]:
products_df.to_csv('products_df.csv',index=False,encoding="utf-8")

In [62]:
reviews_df.to_csv('reviews_df.csv',index=False,encoding="utf-8")